In [2]:
!python -m pip install transformers


  Using cached click-8.0.3-py3-none-any.whl (97 kB)


You should consider upgrading via the 'c:\.virtualenv\sequence-processing\Scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
from transformers import BertModel

dnabert_3_pretrained = './pretrained/3-new-12w-0'

model = BertForMaskedLM.from_pretrained(dnabert_3_pretrained)


In [9]:
model.bert



BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(69, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
 

In [ ]:
from torch import nn
class MTDNABert(nn.Module):
    def __init__(self, complete_bert_model, promoter_head, splice_site_head, polya_head):
        super().__init__()
        self.bert = model.bert
        self.promoter = promoter_head
        self.splice_site = splice_site_head
        self.polya = polya_head

    def forward(self, x):
        x = self.bert(x)
        x_prom = self.promoter(x)
        x_splice_site = self.splice_site(x)
        x_polya = self.polya(x)